<a href="https://colab.research.google.com/github/t-ben2/datasci_python_primer/blob/master/EnronSpamHam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This directory contains the Enron-Spam datasets, as described in the 
paper:

V. Metsis, I. Androutsopoulos and G. Paliouras, "Spam Filtering with 
Naive Bayes - Which Naive Bayes?". Proceedings of the 3rd Conference 
on Email and Anti-Spam (CEAS 2006), Mountain View, CA, USA, 2006.

The "preprocessed" subdirectory contains the messages in the 
preprocessed format that was used in the experiments of the paper.
Each message is in a separate text file. The number at the beginning
of each filename is the "order of arrival".

The "raw" subdirectory contains the messages in their original form. 
Spam messages in non-Latin encodings, ham messages sent by the owners 
of the mailboxes to themselves (sender in "To:", "Cc:", or "Bcc" 
field), and a handful of virus-infected messages have been removed, 
but no other modification has been made. The messages in the "raw" 
subdirectory are more than the corresponding messages in the 
"preprocessed" subdirectory, because: (a) duplicates are preserved 
in the "raw" form, and (b) during the preprocessing, ham and/or spam 
messages were randomly subsampled to obtain the desired ham:spam 
ratios. See the paper for further details.

The Enron-Spam datasets are available from: 
<http://www.iit.demokritos.gr/skel/i-config/> and
<http://www.aueb.gr/users/ion/publications.html>.

The paper is available from:
<http://www.ceas.cc/> and 
<http://www.aueb.gr/users/ion/publications.html>.

V. Metsis, I. Androutsopoulos and G. Paliouras  

This file last updated: June 19, 2006.

1. Acquire and explore the files.
2. Load the data into Python.
3. Explore the data.
4. Learn to process the data using NLTK.
5. Split the data between train and test.
6. Implement the classifier
7. Explore and visualize the results.

In [336]:
import numpy as np
import pandas as pd

import os

# Import Data

In [337]:
spam_dir = './enron_data/raw_spam'
ham_dir = './enron_data/raw_ham'

In [338]:
#List all files in directory using python 'os' library
os.listdir('./enron_data')

['.DS_Store', 'raw_ham', 'preprocessed_files', 'raw_spam']

In [339]:
#List all files/folders in directory using iPython magic commands
#Allows us to interact directly with the command line
folders = !ls ./enron_data
folders

['preprocessed_files', 'raw_ham', 'raw_spam']

In [340]:
files = !ls ./enron_data/preprocessed_files
files

['enron1',
 'enron1.tar.gz',
 'enron2',
 'enron2.tar.gz',
 'enron3',
 'enron3.tar.gz',
 'enron4',
 'enron4.tar.gz',
 'enron5',
 'enron5.tar.gz',
 'enron6',
 'enron6.tar.gz']

In [341]:
#View shell command we are running
!cat ./bash_scripts/unzip_tar_gz.sh

#!/bin/sh
for file in /Users/tbennett/datasci_python_primer/enron_data/$1/*.gz 
do 
tar -xf "$file" -C /Users/tbennett/datasci_python_primer/enron_data/$1;
echo "Succesfully unzipped" "$file"
done


In [342]:
for arg in ['raw_ham','raw_spam','preprocessed_files']:
    os.system("./bash_scripts/unzip_tar_gz.sh '%s'" % arg)

In [343]:
%%bash
cd ./bash_scripts/
bash unzip_tar_gz.sh raw_ham


Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/beck-s.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/farmer-d.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/kaminski-v.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/kitchen-l.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/lokay-m.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/williams-w3.tar.gz


In [344]:
import subprocess
shell_script = [subprocess.run(["./bash_scripts/unzip_tar_gz.sh", "%s" % arg], stdin=subprocess.PIPE, stdout=subprocess.PIPE) for arg in ['raw_ham','raw_spam','preprocessed_files']]

In [345]:
print(shell_script[0].stdout.decode(),'\n') #decode converts output from bytes to string for readability
print(shell_script[1].stdout.decode(),'\n')
print(shell_script[2].stdout.decode(),'\n')

Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/beck-s.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/farmer-d.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/kaminski-v.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/kitchen-l.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/lokay-m.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_ham/williams-w3.tar.gz
 

Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_spam/BG.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_spam/GP.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/raw_spam/SH.tar.gz
 

Succesfully unzipped /Users/tbennett/datasci_python_primer/enron_data/preprocessed_files/enron1.tar.gz
Succesfully unzipped /Users/tbennett/datasci_python_prime

In [346]:
#Blue items listed are folders
!ls ./enron_data/raw_spam

BG        BG.tar.gz GP        GP.tar.gz SH        SH.tar.gz


In [347]:
!ls ./enron_data/raw_spam/BG/2004/08

1091394468.23940_19.txt   1092075793.29722_464.txt  1092891125.24437_638.txt
1091394468.23940_21.txt   1092075794.29722_502.txt  1092891131.24437_647.txt
1091394475.23940_40.txt   1092075795.29722_552.txt  1092891132.24437_653.txt
1091394514.25221_6.txt    1092075795.29722_554.txt  1092891136.24437_671.txt
1091394516.25221_8.txt    1092075806.29722_608.txt  1092891141.24437_691.txt
1091394636.25221_97.txt   1092075806.29722_628.txt  1092891142.24437_697.txt
1091394644.25221_126.txt  1092075806.29722_642.txt  1092891144.24437_713.txt
1091394675.25221_203.txt  1092075806.29722_646.txt  1092891145.24437_715.txt
1091394677.25221_205.txt  1092075806.29722_648.txt  1092891155.24437_732.txt
1091394679.25221_221.txt  1092075807.29722_654.txt  1092891160.24437_781.txt
1091394680.25221_225.txt  1092075807.29722_686.txt  1092891160.24437_783.txt
1091394787.25221_342.txt  1092075807.29722_692.txt  1092891160.24437_785.txt
1091394791.25221_350.txt  1092075811.29722_704.txt  1092891163.24437_817.txt

In [348]:
#Check first text file to inspect data and structure
!cat ./enron_data/raw_spam/BG/2004/08/1091394468.23940_19.txt

Return-Path: <Denny@mailbox.sk>
Delivered-To: rait@bruce-guenter.dyndns.org
Received: (qmail 10573 invoked from network); 31 Jul 2004 18:41:37 -0000
Received: from localhost (localhost [127.0.0.1])
  by bruce-guenter.dyndns.org ([192.168.1.3]); 31 Jul 2004 18:41:37 -0000
Received: from zak.futurequest.net ([127.0.0.1])
  by localhost ([127.0.0.1])
  with SMTP via TCP; 31 Jul 2004 18:41:37 -0000
Received: (qmail 3535 invoked from network); 31 Jul 2004 18:41:36 -0000
Received: from 69.5.6.152 (unknown [221.149.203.7])
  by zak.futurequest.net ([69.5.6.152])
  with SMTP via TCP; 31 Jul 2004 18:41:34 -0000
Received: from 122.22.148.92 by 221.149.203.7; Sat, 31 Jul 2004 23:34:33 +0400
Message-ID: <ZWGIAOGFNYSAGOTRMYLYK@techie.com>
From: "Bertha " <Denny@mailbox.sk>
Reply-To: "Bertha " <Denny@mailbox.sk>
To: rait@bruce-guenter.dyndns.org
Subject: Get it up and keep it up praecox
Date: Sat, 31 Jul 2004 12:39:33 -0700
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--08774934

In [349]:
!ls ./enron_data/raw_ham/williams-w3/hr

1  13 17 20 24 28 31 35 39 42 46 5  53 57 60 64 68 71 75 79 82 86
10 14 18 21 25 29 32 36 4  43 47 50 54 58 61 65 69 72 76 8  83 9
11 15 19 22 26 3  33 37 40 44 48 51 55 59 62 66 7  73 77 80 84
12 16 2  23 27 30 34 38 41 45 49 52 56 6  63 67 70 74 78 81 85


In [350]:
cat  ./enron_data/raw_ham/williams-w3/hr/1
















Hello all!

Thanks again to everyone who participated in our Enron Elves Event! We were able to provide each family a number of gifts for the children as well as a gift certificate to Fred Meyer.

We will be holding the drawing for the Blazer tickets next Monday (January 7th), when more employees are likely to be present. For your reference, here is some additional information about the games:

Package 1:	Blazers vs. Cleveland Cavaliers
		Sunday, January 13th - 7:00 pm
		4 tickets in the lower level, parking included

Package 2:	Blazers vs. Minnesota Timberwolves
		Wednesday, March 6th - 7:00 pm
		4 tickets in the lower level, parking included

If you donated money or a gift and were not able to enter your name into the drawing, please us know--we would be more than happy to add your name.  

Also, we have requested receipts from the Salvation Army for the cash donations that were received. If you would like a receipt for your donation, please contact Grace directly. 

P

In [351]:
!ls ./enron_data/preprocessed_files/enron1/ham

0001.1999-12-10.farmer.ham.txt 2561.2000-10-17.farmer.ham.txt
0002.1999-12-13.farmer.ham.txt 2563.2000-10-17.farmer.ham.txt
0003.1999-12-14.farmer.ham.txt 2564.2000-10-17.farmer.ham.txt
0004.1999-12-14.farmer.ham.txt 2565.2000-10-18.farmer.ham.txt
0005.1999-12-14.farmer.ham.txt 2567.2000-10-18.farmer.ham.txt
0007.1999-12-14.farmer.ham.txt 2569.2000-10-18.farmer.ham.txt
0009.1999-12-14.farmer.ham.txt 2571.2000-10-18.farmer.ham.txt
0010.1999-12-14.farmer.ham.txt 2572.2000-10-18.farmer.ham.txt
0011.1999-12-14.farmer.ham.txt 2573.2000-10-18.farmer.ham.txt
0012.1999-12-14.farmer.ham.txt 2574.2000-10-18.farmer.ham.txt
0013.1999-12-14.farmer.ham.txt 2576.2000-10-18.farmer.ham.txt
0014.1999-12-15.farmer.ham.txt 2577.2000-10-18.farmer.ham.txt
0015.1999-12-15.farmer.ham.txt 2578.2000-10-18.farmer.ham.txt
0016.1999-12-15.farmer.ham.txt 2579.2000-10-18.farmer.ham.txt
0019.1999-12-15.farmer.ham.txt 2582.2000-10-18.farmer.ham.txt
0020.1999-12-15.farmer.ham.txt 2584.2000-10-18.farmer.ham.txt
0021.199

In [352]:
!cat ./enron_data/preprocessed_files/enron1/ham/0001.1999-12-10.farmer.ham.txt
!cat ./enron_data/preprocessed_files/enron1/ham/0002.1999-12-13.farmer.ham.txt

Subject: christmas tree farm pictures
Subject: vastar resources , inc .
gary , production from the high island larger block a - 1 # 2 commenced on
saturday at 2 : 00 p . m . at about 6 , 500 gross . carlos expects between 9 , 500 and
10 , 000 gross for tomorrow . vastar owns 68 % of the gross production .
george x 3 - 6992
- - - - - - - - - - - - - - - - - - - - - - forwarded by george weissman / hou / ect on 12 / 13 / 99 10 : 16
am - - - - - - - - - - - - - - - - - - - - - - - - - - -
daren j farmer
12 / 10 / 99 10 : 38 am
to : carlos j rodriguez / hou / ect @ ect
cc : george weissman / hou / ect @ ect , melissa graves / hou / ect @ ect
subject : vastar resources , inc .
carlos ,
please call linda and get everything set up .
i ' m going to estimate 4 , 500 coming up tomorrow , with a 2 , 000 increase each
following day based on my conversations with bill fischer at bmar .
d .
- - - - - - - - - - - - - - - - - - - - - - forwarded by daren j farmer / hou / ect on 12 / 10 / 99 10 : 34
am

# NLP & Feature Extraction

In [353]:
file = open('./enron_data/preprocessed_files/enron1/ham/0002.1999-12-13.farmer.ham.txt','r')
print(file.read())
file.close()

Subject: vastar resources , inc .
gary , production from the high island larger block a - 1 # 2 commenced on
saturday at 2 : 00 p . m . at about 6 , 500 gross . carlos expects between 9 , 500 and
10 , 000 gross for tomorrow . vastar owns 68 % of the gross production .
george x 3 - 6992
- - - - - - - - - - - - - - - - - - - - - - forwarded by george weissman / hou / ect on 12 / 13 / 99 10 : 16
am - - - - - - - - - - - - - - - - - - - - - - - - - - -
daren j farmer
12 / 10 / 99 10 : 38 am
to : carlos j rodriguez / hou / ect @ ect
cc : george weissman / hou / ect @ ect , melissa graves / hou / ect @ ect
subject : vastar resources , inc .
carlos ,
please call linda and get everything set up .
i ' m going to estimate 4 , 500 coming up tomorrow , with a 2 , 000 increase each
following day based on my conversations with bill fischer at bmar .
d .
- - - - - - - - - - - - - - - - - - - - - - forwarded by daren j farmer / hou / ect on 12 / 10 / 99 10 : 34
am - - - - - - - - - - - - - - - - - - -

In [354]:
def text_extractor(filepath):
    
    '''Takes in a filepath containing multiple text files
    and outputs a dictionary containing the filename and an array of features for each'''
    
    feature_dict ={}
    list_of_files = os.listdir(filepath)
    
    for filename in list_of_files:

        file = open(filepath + filename,'r',encoding='utf-8',errors='ignore')
        features = file.readlines() #Creates a list where each line is one item
        feature_dict[filename] = features #Create a dictionary with the filename, followed by list of features
        file.close() #Best practice to explicity close file once used
        
    return feature_dict

In [355]:
ham_features = text_extractor('./enron_data/preprocessed_files/enron1/ham/')
spam_features = text_extractor('./enron_data/preprocessed_files/enron1/spam/')

In [356]:
#Inspect first spam entry
dict_iterator = iter(spam_features.values())
next(dict_iterator)

['Subject: what up , , your cam babe\n',
 'what are you looking for ?\n',
 "if your looking for a companion for friendship , love , a date , or just good ole '\n",
 'fashioned * * * * * * , then try our brand new site ; it was developed and created\n',
 "to help anyone find what they ' re looking for . a quick bio form and you ' re\n",
 'on the road to satisfaction in every sense of the word . . . . no matter what\n',
 'that may be !\n',
 'try it out and youll be amazed .\n',
 'have a terrific time this evening\n',
 'copy and pa ste the add . ress you see on the line below into your browser to come to the site .\n',
 'http : / / www . meganbang . biz / bld / acc /\n',
 'no more plz\n',
 'http : / / www . naturalgolden . com / retract /\n',
 'counterattack aitken step preemptive shoehorn scaup . electrocardiograph movie honeycomb . monster war brandywine pietism byrne catatonia . encomia lookup intervenor skeleton turn catfish .\n']

In [357]:
#Inspect first ham entry
dict_iterator = iter(ham_features.values())
next(dict_iterator)

['Subject: ena sales on hpl\n',
 "just to update you on this project ' s status :\n",
 'based on a new report that scott mills ran for me from sitara , i have come up\n',
 'with the following counterparties as the ones to which ena is selling gas off\n',
 "of hpl ' s pipe .\n",
 'altrade transaction , l . l . c . gulf gas utilities company\n',
 'brazoria , city of panther pipeline , inc .\n',
 'central illinois light company praxair , inc .\n',
 'central power and light company reliant energy - entex\n',
 'ces - equistar chemicals , lp reliant energy - hl & p\n',
 'corpus christi gas marketing , lp southern union company\n',
 'd & h gas company , inc . texas utilities fuel company\n',
 'duke energy field services , inc . txu gas distribution\n',
 'entex gas marketing company union carbide corporation\n',
 'equistar chemicals , lp unit gas transmission company inc .\n',
 "since i ' m not sure exactly what gets entered into sitara , pat clynes\n",
 "suggested that i check with daren farm

In [358]:
#No. files extracted into each dictionary
print('ham:',len(ham_features.keys()))
print('spam:',len(spam_features.keys()))

ham: 3672
spam: 1500


In [359]:
#Combine both dictionaries
#**dict1 & **dict2 expands the contents of both the dictionaries to a collection of key value pairs
all_features = {**ham_features, **spam_features} 

len(all_features)

5172

In [360]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [361]:
stemmer.stem('running') #Example of word being reduced to its root stem

'run'

In [362]:
# define a function that accepts text and returns a list of lemmas
def word_tokenize(text, how='lemma'):
    words = text.split(' ')  # tokenize into words
    return [stemmer.stem(word) for word in words]

In [363]:
from sklearn.feature_extraction.text import CountVectorizer

In [364]:
vect = CountVectorizer(min_df = 0.05,analyzer = word_tokenize) #only includes words that occur in at least 5% of the corpus documents
_ = vect.fit_transform(ham_features['1061.2000-05-10.farmer.ham.txt'])
_.shape

(46, 48)

https://dziganto.github.io/Sparse-Matrices-For-Efficient-Machine-Learning/

In [365]:
#a TfidfVectorizer is the same as CountVectorizer, in that it constructs features from tokens, 
#but it takes a step further and normalizes counts to frequency of occurrences across a corpus.

from sklearn.feature_extraction.text import TfidfVectorizer

In [375]:
tfidf_vect = TfidfVectorizer(min_df = 0.05,analyzer = word_tokenize) #only includes words that occur in at least 5% of the corpus documents
X = tfidf_vect.fit_transform(ham_features['1061.2000-05-10.farmer.ham.txt'])
X.shape

AttributeError: 'list' object has no attribute 'split'

In [367]:
len(tfidf_vect.get_feature_names())

48

In [368]:
print(X[0])

  (0, 16)	0.5146654414277028
  (0, 36)	0.699776963670117
  (0, 33)	0.49541062212740233


In [373]:
y = [filename.split('.')[-2] for filename in list(all_features.keys())]
len(y)

5172

In [372]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train,y_train)

classifier.predict(X_test)

ValueError: Found input variables with inconsistent numbers of samples: [46, 5172]